### Poisson Matrix Factorization Evaluation
This workbook is to test the Probabilistic factor model (Poisson MF)

In [1]:
from poismf import PoisMF
import pandas as pd
import numpy as np
import pickle
from geopy.distance import geodesic

dir = 'E:\Sebnewrepo\Rec_sys_lab\Geo/'
user_item_count = 'user_item_count.pkl'
last_n_center = 'last_n_center.pkl'
checkin_location = 'checkin_location.pkl'

In [2]:
def load_pickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f, encoding='latin1')

In [3]:
user_records = load_pickle(dir + user_item_count)
user_records.head()

,count,location_encode,user_encode
6542,1,151,0
6543,2,871,0
6544,1,1117,0
6545,1,1279,0
6546,1,2150,0


In [4]:
user_records.rename(columns = {'user_encode':'UserId','location_encode':'ItemId', 'count': 'Count'}, inplace = True)
user_records.head()

,Count,ItemId,UserId
6542,1,151,0
6543,2,871,0
6544,1,1117,0
6545,1,1279,0
6546,1,2150,0


In [5]:
user_records[user_records['UserId'] == 1].sort_values(by = ['Count'])

,Count,ItemId,UserId
8983,1,520,1
9008,1,96386,1
9009,1,96394,1
9010,1,96611,1
9011,1,101088,1
9012,1,101446,1
9013,1,102134,1
9014,1,103139,1
9016,1,106583,1
9017,1,107775,1


#### Train Test Split

In [6]:
from sklearn.model_selection import train_test_split
from copy import deepcopy

In [7]:
df_train, df_test = train_test_split(user_records, test_size=.2)
df_train = df_train.copy()
users_train = np.unique(df_train.UserId.to_numpy())
items_train = np.unique(df_train.ItemId.to_numpy())
df_test = df_test.loc[df_test.UserId.isin(users_train) &
                      df_test.ItemId.isin(items_train)]
df_train["UserId"] = pd.Categorical(df_train.UserId, users_train).codes
df_train["ItemId"] = pd.Categorical(df_train.ItemId, items_train).codes
df_test["UserId"] = pd.Categorical(df_test.UserId, users_train).codes
df_test["ItemId"] = pd.Categorical(df_test.ItemId, items_train).codes
users_test = np.unique(df_test.UserId.to_numpy())

print("Number of entries in training data: {:,}".format(df_train.shape[0]))
print("Number of entries in test data: {:,}".format(df_test.shape[0]))
print("Number of users in training data: {:,}".format(users_train.shape[0]))
print("Number of users in test data: {:,}".format(users_test.shape[0]))
print("Number of items in training and test data: {:,}".format(items_train.shape[0]))

Number of entries in training data: 117,889
Number of entries in test data: 8,353
Number of users in training data: 5,000
Number of users in test data: 3,341
Number of items in training and test data: 95,249


#### Ranking Metrics for Evaluation

In [8]:
from sklearn.metrics import roc_auc_score
from joblib import Parallel, delayed

from sklearn.metrics import roc_auc_score
from joblib import Parallel, delayed

## Note: this is a computationally inefficient implementation of the
## test metrics, not recommended to use outside of this notebook
def print_ranking_metrics(A, B, df_train, df_test, users_test,
                          nusers=1000, top_n=5, seed=1,
                          njobs=-1):
    
    n_users = A.shape[0]
    n_items = B.shape[0]
    rng = np.random.default_rng(seed=seed)
    chosen_users = rng.choice(users_test, size=nusers, replace=False)
    all_train = df_train.loc[df_train.UserId.isin(chosen_users)]
    all_test = df_test.loc[df_test.UserId.isin(chosen_users)]
    
    def metrics_single_user(user):
        # item in the test set
        ypos = all_test.ItemId.loc[all_test.UserId == user].to_numpy()
        # item in the train set
        ytrain = all_train.ItemId.loc[all_train.UserId == user].to_numpy()
        # item not in the train and test set - neglect set
        yneg = np.setdiff1d(np.arange(n_items), np.r_[ypos, ytrain])
        # the test item should be in neglect set and test set
        ytest = np.r_[yneg, ypos]
        # the first yneg-th items are from neglect set, the last ypos-th items are from test set
        yhat = B[ytest].dot(A[user])
        auc = roc_auc_score(np.r_[np.zeros(yneg.shape[0]),
                                  np.ones(ypos.shape[0])],
                            yhat)
        # if the index of the top Ns are in last ypos-th index, this rec is a successful rec
        topN = np.argsort(-yhat)[:top_n]
        p_at_k = np.mean(topN >= yneg.shape[0])
        p_at_k_rnd = ypos.shape[0] / ytest.shape[0] ## <- baseline
        return auc, p_at_k, p_at_k_rnd

    res_triplets = Parallel(n_jobs = njobs)\
                    (delayed(metrics_single_user)(u) \
                        for u in chosen_users)

    res_triplets = np.array(res_triplets)
    auc = np.mean(res_triplets[:,0])
    p_at_k = np.mean(res_triplets[:,1])
    p_at_k_rnd = np.mean(res_triplets[:,2])
    print("AUC: %.4f [random: %.2f]" % (auc, 0.5))
    print("P@%d: %.4f [random: %.4f]" % (top_n,
                                         p_at_k,
                                         p_at_k_rnd))

#### Model Fitting

In [34]:
%%time
model_balanced = PoisMF(reindex=False, method="cg", use_float=False,
                        k=200, niter=50, maxupd=25, l2_reg=1e4)\
                    .fit(df_train)

Wall time: 14.2 s


In [35]:
#Precision of top 5
print_ranking_metrics(model_balanced.A, model_balanced.B,
                      df_train, df_test, users_test, top_n=5)

AUC: 0.9053 [random: 0.50]
P@5: 0.0230 [random: 0.0000]


In [36]:
#Precision of top 10
print_ranking_metrics(model_balanced.A, model_balanced.B,
                      df_train, df_test, users_test, top_n=10)

AUC: 0.9053 [random: 0.50]
P@10: 0.0158 [random: 0.0000]


### PMF with Geo Factors
The distance factor for the final prediction matrix is:
$ \Bigg[\frac{{d}_{0}}{{d}_{0} + {d(i,j)}}\Bigg]^\tau $  

So. the equation for final prediction matirx is :
$ {f}_{i,j} = {u}^{T}_{i}{v}_{j} * \Bigg[\frac{{d}_{0}}{{d}_{0} + {d(i,j)}}\Bigg]^\tau $

${d}_{0}$ is the user daily activity range: lets assume ${d}_{0} = 100km $

In [37]:
# load last-n center file and checkin
n_center = load_pickle(dir + last_n_center)
location = load_pickle(dir + checkin_location)

In [38]:
n_center.head()

,latitude,longitude
user_mapping,,
0,30.376921,-97.756796
1,47.823526,9.887863
2,28.373947,-81.536761
3,51.097721,-0.936662
4,30.315782,-97.732966


In [39]:
from math import radians, cos, sin, asin, sqrt
def geodistance(lng1, lat1, lng2, lat2):
    
    lng1, lat1, lng2, lat2 = map(radians, [float(lng1), float(lat1), float(lng2), float(lat2)])
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2 
    distance=2*asin(sqrt(a))*6371*1000
    distance=round(distance/1000,3)
    return distance



In [53]:
from sklearn.metrics import roc_auc_score
from joblib import Parallel, delayed

from sklearn.metrics import roc_auc_score
from joblib import Parallel, delayed

## Note: this is a computationally inefficient implementation of the
## test metrics, not recommended to use outside of this notebook
def print_ranking_metrics_GEO(A, B, df_train, df_test, users_test, location,
                          nusers=1000, top_n=5, seed=1,
                          njobs=-1):
    
    n_users = A.shape[0]
    n_items = B.shape[0]
    rng = np.random.default_rng(seed=seed)
    chosen_users = rng.choice(users_test, size=nusers, replace=False)
    all_train = df_train.loc[df_train.UserId.isin(chosen_users)]
    all_test = df_test.loc[df_test.UserId.isin(chosen_users)]
    
    def metrics_single_user_GEO(user, location):
        # item in the test set
        ypos = all_test.ItemId.loc[all_test.UserId == user].to_numpy()
        # item in the train set
        ytrain = all_train.ItemId.loc[all_train.UserId == user].to_numpy()
        # item not in the train and test set - neglect set
        yneg = np.setdiff1d(np.arange(n_items), np.r_[ypos, ytrain])
        # the test item should be in neglect set and test set
        ytest = np.r_[yneg, ypos]
        # the first yneg-th items are from neglect set, the last ypos-th items are from test set
        yhat = B[ytest].dot(A[user])
        auc = roc_auc_score(np.r_[np.zeros(yneg.shape[0]),
                                  np.ones(ypos.shape[0])],
                            yhat)
        # multiply with geo factor
        check = location.loc[ytest][:]
        dist = check[['latitude','longitude']]\
               .apply(lambda x: 1000/(1000+geodistance(x['longitude'],
                                            x['latitude'],
                                            user_actCenter['longitude'],
                                            user_actCenter['latitude'])),axis=1)
        m = yhat * dist.to_numpy()
        
        # if the index of the top Ns are in last ypos-th index, this rec is a successful rec
        
        topN = np.argsort(-m)[:top_n]
        p_at_k = np.mean(topN >= yneg.shape[0])
        p_at_k_rnd = ypos.shape[0] / ytest.shape[0] ## <- baseline
        return auc, p_at_k, p_at_k_rnd

    res_triplets = Parallel(n_jobs = njobs)\
                    (delayed(metrics_single_user_GEO)(u, location) \
                        for u in chosen_users)

    res_triplets = np.array(res_triplets)
    auc = np.mean(res_triplets[:,0])
    p_at_k = np.mean(res_triplets[:,1])
    p_at_k_rnd = np.mean(res_triplets[:,2])
    print("AUC: %.4f [random: %.2f]" % (auc, 0.5))
    print("P@%d: %.4f [random: %.4f]" % (top_n,
                                         p_at_k,
                                         p_at_k_rnd))

In [55]:
print_ranking_metrics_GEO(model_balanced.A, model_balanced.B,
                      df_train, df_test, users_test, location, top_n=10)

AUC: 0.9053 [random: 0.50]
P@10: 0.0073 [random: 0.0000]


#### Test Mode

In [41]:
n_users = model_balanced.A.shape[0]
n_items = model_balanced.B.shape[0]
rng = np.random.default_rng(seed=1)
chosen_users = rng.choice(users_test, size=1000, replace=False)
all_train = df_train.loc[df_train.UserId.isin(chosen_users)]
all_test = df_test.loc[df_test.UserId.isin(chosen_users)]

In [42]:
all_test

,Count,ItemId,UserId
3398335,1,2444,3721
3691098,1,11059,4286
3329980,1,27638,3628
3442507,1,8581,3797
1268361,1,1603,591
...,...,...,...
3730104,1,44967,4368
2686543,2,8967,2710
742593,1,1086,202
828721,1,7816,249


In [44]:
ypos = all_test.ItemId.loc[all_test.UserId == 3628].to_numpy()
ytrain = all_train.ItemId.loc[all_train.UserId == 3628].to_numpy()
yneg = np.setdiff1d(np.arange(n_items), np.r_[ypos, ytrain])
ytest = np.r_[yneg, ypos]
yhat = model_balanced.B[ytest].dot(model_balanced.A[3628])

In [45]:
ytest

array([    0,     1,     2, ..., 95248, 27638, 20750])

In [47]:
user_actCenter = n_center.loc[3628][:]
user_actCenter

latitude     36.653916
longitude    37.718318
Name: 3628, dtype: float64

In [48]:
%%time
ttt = location.loc[ytest][['latitude','longitude']]
dist = ttt[['latitude','longitude']]\
               .apply(lambda x: 100/(100+geodistance(x['longitude'],
                                            x['latitude'],
                                            user_actCenter['longitude'],
                                            user_actCenter['latitude'])),axis=1)
dist.shape

Wall time: 1.57 s


(95218,)

In [49]:
dist

0        0.009029
1        0.008600
2        0.009035
3        0.009025
4        0.009036
           ...   
95246    0.010361
95247    0.031430
95248    0.010241
27638    0.008363
20750    0.036111
Length: 95218, dtype: float64

In [50]:
yhat

array([1.00837026e-06, 8.03237702e-05, 0.00000000e+00, ...,
       0.00000000e+00, 1.18829821e-06, 7.59651378e-06])

In [222]:
m = yhat * dist.to_numpy()

In [223]:
np.argsort(-m)[:5]

array([   307,    171, 941724,   6444,    156], dtype=int64)